## Random Scratch Test
This section is to draw random scratches on an image and check how reilient the classifier is to these randomly introduced noises.
To introduce the random scratches, we basically plot a straight line and some dots on an image using the matplotlib. 
The code is inspired from this [page](https://www.geeksforgeeks.org/plot-a-point-or-a-line-on-an-image-with-matplotlib/). 

In [2]:
# STDLIB
import os
import typing
import csv
# Packages
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from matplotlib import image
import random
from skimage.metrics import structural_similarity as ssim
from tqdm import tqdm

In [186]:
DATA_DIR = "GTSRB"
TRAINING_DATA_PATH = os.path.join(DATA_DIR, "Final_Training/Images")
CORRUPTED_DATA_PATH = os.path.join(DATA_DIR, "corrupted_Data/Images")

In [4]:
def minimum_distance(p1, p2):

    i1, i2 = int(p1[0]), int(p2[0])

    # if both x-coordinates are floats and they have the same integer part
    if i1 != p1[0] and i2 != p2[0] and i1 == i2:

        # find the decimal parts
        d1, d2 = p1[0] - i1, p2[0] - i2

        # find the smaller "C"
        x = min(d1 + d2, (1-d1) + (1-d2))

        # add the y distance to the "C" distance
        return abs(p1[1] - p2[1]) + x

    # repeat with the "y-coordinates are floats" case
    i1, i2 = int(p1[1]), int(p2[1])
    if i1 != p1[1] and i2 != p2[1] and i1 == i2:
        d1, d2 = p1[1] - i1, p2[1] - i2
        y = min(d1 + d2, (1-d1) + (1-d2))
        return abs(p1[0] - p2[0]) + y

    # simple case, return the Manhattan distance
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

In [197]:
image_data = image.imread('10945.ppm')
input_image = image.imread('10945.ppm')

image_height = image_data.shape[0]
image_width = image_data.shape[1]
min_length = round(0.25 * image_height)

print("Image_height = ", image_height, "min_length = ", min_length)

Image_height =  59 min_length =  15


In [205]:

def insert_lines(image) :
    image_data = image
    unmodified_input_image = image_data
    
    #understand the image dimensions
    image_height = image_data.shape[0]
    image_width = image_data.shape[1]
    
    #define the min legth of the line to be drawn on the image
    min_length = round(0.25 * image_height)

    line_length = 0
    iter = 0 
    while (line_length < min_length) and (iter < 10) :
        x = [random.randint(0, image_height), random.randint(0, image_width)]
        y = [random.randint(0, image_height), random.randint(0, image_width)]
        line_length = minimum_distance(x, y)
        #print("The minimum distance is : ", line_length)
        iter = iter + 1
        #print("iter = ", iter)

    random_col = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    #plt.plot(x, y, c=random_col, linewidth=2)
    #plt.imshow(data)
    #plt.show()
    #cv.imwrite('output.ppm', plt)
    #print("hello world")

    cv.line(image_data, x, y, random_col, 1)

    output_image = cv.addWeighted(image_data, 0.15, unmodified_input_image, 0.85, 0)

    #cv.imshow("Image", input_image)
    #cv.imwrite('output.ppm', output_image)

    #print(ssim(np.array(image_data), np.array(unmodified_input_image), multichannel=True))
    return output_image



In [212]:
def readTrafficSigns(rootpath: str) -> typing.Tuple[np.array, np.array]:
    '''Reads traffic sign data for German Traffic Sign Recognition Benchmark.

    Arguments: path to the traffic sign data, for example './GTSRB/Training'
    Returns:   list of images, list of corresponding labels'''
    images = [] # images
    processed_images = []   # array to hold the processed images
    labels = [] # corresponding labels
    # loop over all 42 classes
    for c in tqdm(range(0,43)):
        prefix = rootpath + '/' + format(c, '05d') + '/' # subdirectory for class
        with open(prefix + 'GT-'+ format(c, '05d') + '.csv') as gtFile:
            gtReader = csv.reader(gtFile, delimiter=';') # csv parser for annotations file
            next(gtReader) # skip header
            # loop over all images in current annotations file
            for row in tqdm(gtReader):
                #image_filename = row[0]
                print("File Path = ", (prefix + row[0]))
                image = cv.imread(prefix + row[0]) # read image, the 1th column is the filename
                image = cv.resize(image, (32,32)) # resize to 32x32
                # image = cv.cvtColor(image, cv.COLOR_BGR2GRAY) # convert to grayscale
                processed_image = insert_lines(image)
                corrupted_output_filename = "corrupted_" + row[0]
                cv.imwrite(prefix + corrupted_output_filename, processed_image)
                images.append(image)
                labels.append(row[7]) # the 8th column is the label
    # convert the list to numpy arrays
    images = np.array(images).astype(np.float32) # this allows us to convert it to a tensor
    images = images/255 # Everyone seems to do this so we will too!!!
    labels = np.array(labels).astype(np.float32)
    return images, labels

images, labels = readTrafficSigns(TRAINING_DATA_PATH)
assert len(images) == len(labels)


NameError: name 'tqdm' is not defined